In [ ]:
import pandas as pd

train_df = pd.read_csv("../data/train.csv")
train_df.head()

In [ ]:
import pandas as pd
import numpy as np

oof_df = pd.DataFrame()

for fold_idx in range(0,10):
    print(fold_idx)
    data = np.load(f"../ckpt/baseline_fold_{fold_idx}/valid_fold{fold_idx}_step3417.npy")
    label = np.load(f"../ckpt/baseline_fold_{fold_idx}/valid_fold{fold_idx}_step3417_label.npy").astype(int)
    
    #assert label is ascending order
    assert np.all(np.diff(label) >= 0), "Labels are not in ascending order"
    
    data = data.flatten()
    data = data[data != -100]
    
    cur_valid_idx_list = kfold_dict[fold_idx][1]
    
    cur_valid = train_df.iloc[cur_valid_idx_list]
    cur_valid_expanded = (
        cur_valid.assign(text=cur_valid["full_text"].str.split("\n"))  # 새 column 'text'에 리스트 저장
        .explode("text")                              # 리스트 → 여러 행
        .drop(columns="full_text")                    # 원본 full_text 제거
        .reset_index(drop=True)                       # 인덱스 리셋
    )

    cur_valid_expanded['logits'] = data
    cur_valid_expanded['probs'] = sigmoid(cur_valid_expanded['logits'])
    
    oof_df = pd.concat([oof_df, cur_valid_expanded], ignore_index=True)

In [ ]:
oof_df.rename(columns={'text': 'full_text'}, inplace=True)
oof_df.loc[oof_df['generated']==0, 'probs'] = 0.0
oof_df['generated'] = oof_df['probs']

In [ ]:
oof_df.to_csv("../data_oof/train.csv", index=False)